<a href="https://colab.research.google.com/github/santiagogr10/neural-networks-exercises/blob/main/insignia1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Santiago García Rincón**



El conjunto de datos **MNIST (Modified National Institute of Standards and Technology)** es un benchmark ampliamente utilizado en aprendizaje automático y visión por computadora. Contiene 70,000 imágenes en escala de grises de dígitos escritos a mano (60,000 para entrenamiento y 10,000 para prueba), cada una con una resolución de 28x28 píxeles. Es ideal para tareas de clasificación, ya que las imágenes están etiquetadas del 0 al 9. Su simplicidad y accesibilidad lo convierten en una herramienta esencial para aprender y probar algoritmos de redes neuronales y técnicas avanzadas como redes convolucionales y regularización.

La intención de este cuadernillo es desarrollar un modelo de predicción utilizando redes neuronales convolucionales, abordando un ejercicio clásico en el campo del aprendizaje automático.

In [ ]:
import tensorflow as tf

!pip install mitdeeplearning --quiet
import mitdeeplearning as mdl

import matplotlib.pyplot as plt
import numpy as np
import random
from tqdm import tqdm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 36.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done


## Cargar el conjunto de datos MNSIT

Primero, cargamos el conjunto de datos MNIST desde TensorFlow y lo dividimos en conjuntos de entrenamiento y prueba. También normalizamos las imágenes para que los valores de píxeles estén en el rango [0, 1].

In [ ]:
# Cargar el conjunto de datos MNIST
mnist = tf.keras.datasets.mnist
(train_images, train_labels), (test_images, test_labels) = mnist.load_data()

# Normalizar las imágenes al rango [0, 1]
train_images = train_images / 255.0
test_images = test_images / 255.0

# Expandir la dimensión de las imágenes para adaptarse a las CNNs
train_images = np.expand_dims(train_images, axis=-1)
test_images = np.expand_dims(test_images, axis=-1)

# Verificar las formas de los datos
print("Forma de train_images:", train_images.shape)
print("Forma de train_labels:", train_labels.shape)
print("Forma de test_images:", test_images.shape)
print("Forma de test_labels:", test_labels.shape)


 9601024/11490434 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


11490434/11490434 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
Forma de train_images: (60000, 28, 28, 1)
Forma de train_labels: (60000,)
Forma de test_images: (10000, 28, 28, 1)
Forma de test_labels: (10000,)


## Construir la red convolucional

Definimos una CNN con capas de convolución, max pooling, y densas.

In [ ]:
def build_cnn():
    model = tf.keras.Sequential([
        tf.keras.layers.Conv2D(32, (3, 3), activation="relu", input_shape=(28, 28, 1)),  # Convolución
        tf.keras.layers.MaxPooling2D((2, 2)),  # Max pooling
        tf.keras.layers.Conv2D(64, (3, 3), activation="relu"),  # Convolución
        tf.keras.layers.MaxPooling2D((2, 2)),  # Max pooling
        tf.keras.layers.Flatten(),  # Aplanar las características
        tf.keras.layers.Dense(128, activation="relu"),  # Capa totalmente conectada
        tf.keras.layers.Dropout(0.5),  # Regularización con Dropout
        tf.keras.layers.Dense(10, activation="softmax")  # Capa de salida
    ])
    return model

# Construir y resumir el modelo
cnn_model = build_cnn()
cnn_model.summary()


/usr/local/lib/python3.10/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                      │ (None, 26, 26, 32)          │             320 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d (MaxPooling2D)         │ (None, 13, 13, 32)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_1 (Conv2D)                    │ (None, 11, 11, 64)          │          18,496 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_1 (MaxPooling2D)       │ (None, 5, 5, 64)            │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten (Flatten)                    │ (None, 1600)                │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 128)                 │         204,928 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ (None, 128)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 10)                  │           1,290 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 225,034 (879.04 KB)

 Trainable params: 225,034 (879.04 KB)

 Non-trainable params: 0 (0.00 B)

La parte de arriba se ve

## Compilar y entrenar el modelo ( evaluarlo con diferenttes parametros )

Compilamos el modelo con un optimizador y función de pérdida adecuados. Entrenamos el modelo en los datos de entrenamiento. Experimenté con distintos optimizadores y adam fue el que mejor rendiiento tuvo :)

In [ ]:
cnn_model.compile(
    optimizer="sgd",
    loss="sparse_categorical_crossentropy",
    metrics=["accuracy"]
)

history = cnn_model.fit(
    train_images, train_labels,
    epochs=10,
    validation_data=(test_images, test_labels),
    verbose=1
)


Epoch 1/10
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 87s 46ms/step - accuracy: 0.6116 - loss: 1.1825 - val_accuracy: 0.9481 - val_loss: 0.1857
Epoch 2/10
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 57s 31ms/step - accuracy: 0.9242 - loss: 0.2556 - val_accuracy: 0.9666 - val_loss: 0.1100
Epoch 3/10
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 82s 31ms/step - accuracy: 0.9475 - loss: 0.1741 - val_accuracy: 0.9742 - val_loss: 0.0828
Epoch 4/10
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 56s 30ms/step - accuracy: 0.9584 - loss: 0.1372 - val_accuracy: 0.9791 - val_loss: 0.0692
Epoch 5/10
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 82s 30ms/step - accuracy: 0.9649 - loss: 0.1193 - val_accuracy: 0.9824 - val_loss: 0.0588
Epoch 6/10
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 81s 30ms/step - accuracy: 0.9679 - loss: 0.1072 - val_accuracy: 0.9820 - val_loss: 0.0574
Epoch 7/10
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 82s 30ms/step - accuracy: 0.9718 - loss: 0.0950 - val_accuracy: 0.9829 - val_loss: 0.0553
Epoch 8/10
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 56s 30ms/step - accuracy: 0.9743 -

In [ ]:
from tensorflow.keras.optimizers import Adam

# Compile the model with Adam optimizer and lower learning rate
cnn_model.compile(
    optimizer=Adam(learning_rate=0.0001),
    loss="sparse_categorical_crossentropy",
    metrics=["accuracy"]
)

# Train the model
history = cnn_model.fit(
    train_images, train_labels,
    epochs=10,
    validation_data=(test_images, test_labels),
    verbose=1
)


Epoch 1/10
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 60s 32ms/step - accuracy: 0.9763 - loss: 0.0763 - val_accuracy: 0.9877 - val_loss: 0.0398
Epoch 2/10
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 81s 31ms/step - accuracy: 0.9790 - loss: 0.0691 - val_accuracy: 0.9884 - val_loss: 0.0366
Epoch 3/10
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 83s 32ms/step - accuracy: 0.9823 - loss: 0.0637 - val_accuracy: 0.9891 - val_loss: 0.0360
Epoch 4/10
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 82s 32ms/step - accuracy: 0.9833 - loss: 0.0556 - val_accuracy: 0.9886 - val_loss: 0.0329
Epoch 5/10
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 59s 32ms/step - accuracy: 0.9845 - loss: 0.0534 - val_accuracy: 0.9897 - val_loss: 0.0335
Epoch 6/10
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 82s 32ms/step - accuracy: 0.9868 - loss: 0.0441 - val_accuracy: 0.9902 - val_loss: 0.0301
Epoch 7/10
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 82s 32ms/step - accuracy: 0.9868 - loss: 0.0433 - val_accuracy: 0.9910 - val_loss: 0.0283
Epoch 8/10
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 61s 32ms/step - accuracy: 0.9875 -

In [ ]:
# Compilar el modelo
cnn_model.compile(
    optimizer="adam",
    loss="sparse_categorical_crossentropy",
    metrics=["accuracy"]
)

# Entrenar el modelo
history = cnn_model.fit(
    train_images, train_labels,
    epochs=10,
    validation_data=(test_images, test_labels),
    verbose=1
)


Epoch 1/10
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 60s 31ms/step - accuracy: 0.9693 - loss: 0.1003 - val_accuracy: 0.9864 - val_loss: 0.0391
Epoch 2/10
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 59s 31ms/step - accuracy: 0.9810 - loss: 0.0615 - val_accuracy: 0.9907 - val_loss: 0.0282
Epoch 3/10
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 82s 31ms/step - accuracy: 0.9855 - loss: 0.0501 - val_accuracy: 0.9905 - val_loss: 0.0288
Epoch 4/10
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 57s 31ms/step - accuracy: 0.9870 - loss: 0.0395 - val_accuracy: 0.9913 - val_loss: 0.0305
Epoch 5/10
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 82s 31ms/step - accuracy: 0.9907 - loss: 0.0311 - val_accuracy: 0.9891 - val_loss: 0.0326
Epoch 6/10
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 83s 31ms/step - accuracy: 0.9911 - loss: 0.0290 - val_accuracy: 0.9916 - val_loss: 0.0286
Epoch 7/10
 429/1875 ━━━━━━━━━━━━━━━━━━━━ 42s 29ms/step - accuracy: 0.9935 - loss: 0.0206

## Evaluar el modelo y calcular métricas de clasificación


Calculamos métricas de precisión, recall y F1-score

In [ ]:
from sklearn.metrics import classification_report

# Evaluar el modelo en el conjunto de prueba
test_loss, test_accuracy = cnn_model.evaluate(test_images, test_labels, verbose=0)
print(f"Test Loss: {test_loss:.4f}, Test Accuracy: {test_accuracy:.4%}")

# Generar predicciones
y_pred = cnn_model.predict(test_images)
y_pred_classes = np.argmax(y_pred, axis=1)

# Generar métricas de clasificación
print(classification_report(test_labels, y_pred_classes, target_names=[str(i) for i in range(10)]))


## Visualizar imágenes de prueba con predicciones

Visualizamos ejemplos de predicciones correctas e incorrectas.

In [ ]:
import matplotlib.pyplot as plt

# Visualizar imágenes con predicciones
plt.figure(figsize=(10, 10))
for i in range(9):
    plt.subplot(3, 3, i + 1)
    plt.imshow(test_images[i].squeeze(), cmap="gray")
    plt.title(f"True: {test_labels[i]}, Pred: {y_pred_classes[i]}")
    plt.axis("off")
plt.show()


## conclusion




Los resultados muestran que la red convolucional entrenada alcanza un excelente rendimiento, con una precisión final en el conjunto de prueba del **99.2%** y pérdidas muy bajas en las métricas de entrenamiento y validación. El modelo demuestra una alta capacidad para generalizar, con valores de **precisión, recall y F1-score cercanos a 1.00** en todas las clases. Estas métricas indican que el modelo clasifica de manera efectiva los dígitos escritos a mano, con un equilibrio sobresaliente entre predicciones correctas y errores mínimos. Esto valida la eficacia de la arquitectura empleada y el proceso de entrenamiento.